# Fine-tuning transformer

### Getting the data

#### Train/test data

In [1]:
# !mkdir data
# !mkdir data/history
# !wget --no-check-certificate 'https://drive.usercontent.google.com/download?id=1iqg1FIPfbrZWlung6gZqve1MeQWc0Je4&export=download&authuser=1&confirm=t' -O './data/dataset.csv'

In [2]:
import pandas as pd


df = pd.read_csv('../data/dataset.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22224 entries, 0 to 22223
Data columns (total 15 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   title                                 22224 non-null  object 
 1   company                               22224 non-null  object 
 2   location                              22224 non-null  object 
 3   skills                                14384 non-null  object 
 4   source                                22224 non-null  object 
 5   description_no_numbers_with_skills    22224 non-null  object 
 6   experience_from                       22224 non-null  float64
 7   experience_to_adjusted_10             22224 non-null  float64
 8   description_size                      22224 non-null  int64  
 9   description                           22224 non-null  object 
 10  description_no_numbers                22224 non-null  object 
 11  description_no_

### Extra dependencies

In [6]:
%pip install -U sentence-transformers datasets -qqq

Note: you may need to restart the kernel to use updated packages.


In [4]:
# import nltk


# nltk.download('punkt_tab')

### Service functions

In [8]:
import torch
from torch.utils.data import DataLoader
import numpy as np
import gc
import re
import os
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer, LoggingHandler
from sentence_transformers import models, util, datasets, evaluation, losses
# from numba import cuda


os.environ["WANDB_DISABLED"] = "true"


def memory_cleanup():
    "Clean up memory"
    gc.collect()
    torch.cuda.empty_cache()


def get_sentence_lengths(text):
    "Get number of words in each sentence in the text"
    pattern = r'(?<=[.!?])'
    sentences = re.split(pattern, text)
    
    # remove empty strings
    sentences = [sentence for sentence in sentences if len(sentence) > 0]
    
    # get number of words in each sentence
    sentence_lengths = [len(sentence.split()) for sentence in sentences]
    
    return sentences, sentence_lengths


def set_seed(seed: int) -> None:
    "Set seed for reproducibility"
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


def train_tsdae_bert(model_name, train_sentences):
    """Train a denoising auto-encoder model with BERT model.
    more examples at https://sbert.net/examples/unsupervised_learning/TSDAE/README.html"""
    word_embedding_model = models.Transformer(model_name)
    pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(), "cls")
    model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

    # Create the special denoising dataset that adds noise on-the-fly
    train_dataset = datasets.DenoisingAutoEncoderDataset(train_sentences)
    
    # DataLoader to batch your data
    train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True)
    
    # Use the denoising auto-encoder loss
    train_loss = losses.DenoisingAutoEncoderLoss(
        model, decoder_name_or_path=model_name, tie_encoder_decoder=True,
    )
    
    # Call the fit method
    model.fit(
        train_objectives=[(train_dataloader, train_loss)],
        epochs=1,
        weight_decay=0,
        scheduler="constantlr",
        optimizer_params={"lr": 3e-5},
        show_progress_bar=True,
    )
    
    return model

Display model output

In [9]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import t


def display_metrics_with_ci(history: dict):
    # plot mean and ci for train and test r2 for all seeds and all iterations, averaged over seeds
    seeds = list(history.keys())
    def mean_confidence_interval(data, confidence=0.95):
        n = len(data)
        m, se = np.mean(data), np.std(data) / np.sqrt(n)
        h = se * t.ppf((1 + confidence) / 2, n-1)
        return m, m-h, m+h

    r2_train_values = [history[seed]['train_r2'] for seed in seeds]
    r2_test_values = [history[seed]['test_r2'] for seed in seeds]

    r2_train_values = np.array(r2_train_values)
    r2_test_values = np.array(r2_test_values)

    r2_train_mean = np.mean(r2_train_values, axis=0)
    r2_test_mean = np.mean(r2_test_values, axis=0)

    r2_train_ci = np.array([mean_confidence_interval(r2_train_values[:, i]) for i in range(r2_train_values.shape[1])])
    r2_test_ci = np.array([mean_confidence_interval(r2_test_values[:, i]) for i in range(r2_test_values.shape[1])])

    plt.figure(figsize=(10, 6))
    plt.plot(r2_train_mean[1:], label='train')
    plt.fill_between(range(len(r2_train_mean[1:])), r2_train_ci[1:, 1], r2_train_ci[1:, 2], alpha=0.3)

    plt.plot(r2_test_mean[1:], label='test')
    plt.fill_between(range(len(r2_test_mean[1:])), r2_test_ci[1:, 1], r2_test_ci[1:, 2], alpha=0.3)
    plt.title('Mean R2 by epoch, with 95% CI')
    plt.xlabel('Epoch')
    plt.ylabel('R2')
    plt.legend()
    plt.show()

    mae_test_values = [history[seed]['test_mae'] for seed in seeds]
    rmse_test_values = [history[seed]['test_rmse'] for seed in seeds]

    mae_test_values = np.array(mae_test_values)
    rmse_test_values = np.array(rmse_test_values)

    mae_test_mean = np.mean(mae_test_values, axis=0)
    rmse_test_mean = np.mean(rmse_test_values, axis=0)

    mae_test_ci = np.array([mean_confidence_interval(mae_test_values[:, i]) for i in range(mae_test_values.shape[1])])
    rmse_test_ci = np.array([mean_confidence_interval(rmse_test_values[:, i]) for i in range(rmse_test_values.shape[1])])

    # get an index of the epoch, where the test R2 is the highest
    # get mean and CI for this epoch
    best_epoch = np.argmax(r2_test_mean)
    best_epoch_r2 = r2_test_mean[best_epoch]
    best_epoch_mae = mae_test_mean[best_epoch]
    best_epoch_rmse = rmse_test_mean[best_epoch]
    best_epoch_r2_ci = r2_test_ci[best_epoch]
    best_epoch_mae_ci = mae_test_ci[best_epoch]
    best_epoch_rmse_ci = rmse_test_ci[best_epoch]

    print(f'TEST METRICS FOR THE BEST EPOCH (#{best_epoch+1})')
    print(f'R2: mean = {best_epoch_r2:.4f}, 95% CI = [{best_epoch_r2_ci[1]:.4f}, {best_epoch_r2_ci[2]:.4f}]')
    print(f'MAE: mean = {best_epoch_mae:.4f}, 95% CI = [{best_epoch_mae_ci[1]:.4f}, {best_epoch_mae_ci[2]:.4f}]')
    print(f'RMSE: mean = {best_epoch_rmse:.4f}, 95% CI = [{best_epoch_rmse_ci[1]:.4f}, {best_epoch_rmse_ci[2]:.4f}]')

### Traning-related classes

#### Dataset

##### Baseline dataset

In [10]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset

# Dataset for dual textual features
class DualTextDataset(Dataset):
    def __init__(self, df, text_col_1, text_col_2, targets, tokenizer, max_len1, max_len2):
        print('Creating the dataset...')
        # Pre-tokenize and store inputs
        self.tokenized_texts1 = tokenizer(df[text_col_1].tolist(), max_length=max_len1, padding="max_length", truncation=True, return_tensors="pt")
        self.tokenized_texts2 = tokenizer(df[text_col_2].tolist(), max_length=max_len2, padding="max_length", truncation=True, return_tensors="pt")
        # self.tokenized_texts2 = tokenizer(df[text_col_2].tolist(), max_length=max_len, padding="max_length", truncation=True, return_tensors="pt")
        self.targets = targets.tolist()
        self.tokenizer = tokenizer
        # self.max_len = max_len

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, idx):
        # Return only the slice for idx
        inputs1 = {key: val[idx] for key, val in self.tokenized_texts1.items()}
        inputs2 = {key: val[idx] for key, val in self.tokenized_texts2.items()}
        target = torch.tensor(self.targets[idx], dtype=torch.float)
        return inputs1, inputs2, target


#### Model

##### Single-head BERT with MLP

In [11]:
import torch
import torch.nn as nn
from transformers import AutoModel, AutoTokenizer


# single bert for two text features with MLP head for regression
class SingleBERTWithMLP(nn.Module):
    def __init__(self, config):
        super(SingleBERTWithMLP, self).__init__()
        model_name = config["model_name"]
        # Initialize a single BERT model
        self.bert = AutoModel.from_pretrained(model_name)

        # Define MLP head
        hidden_size = self.bert.config.hidden_size
        mlp_hidden_size = config["mlp_hidden_size"]
        self.mlp = nn.Sequential(
            nn.Linear(2 * hidden_size, mlp_hidden_size),  # Double hidden size for concatenation
            nn.ReLU(),
            nn.Linear(mlp_hidden_size, 1)  # Regression output
        )

    def forward(self, input1, attention_mask1, input2, attention_mask2):
        # Pass both inputs through the same BERT model
        cls1 = self.bert(input_ids=input1, attention_mask=attention_mask1).last_hidden_state[:, 0, :]  # CLS token for input1
        cls2 = self.bert(input_ids=input2, attention_mask=attention_mask2).last_hidden_state[:, 0, :]  # CLS token for input2

        # Concatenate CLS embeddings
        combined_cls = torch.cat([cls1, cls2], dim=-1)  # Shape: [batch_size, 2 * hidden_size]

        # Pass through MLP head
        output = self.mlp(combined_cls)
        return output


##### Double-head BERT with MLP

In [12]:
# two berts for two text features with MLP head for regression
class DualBERTWithMLP(nn.Module):
    def __init__(self, config):
        super(DualBERTWithMLP, self).__init__()
        # Initialize two independent BERT models
        model_name = config['model_name']
        self.bert1 = AutoModel.from_pretrained(model_name)
        self.bert2 = AutoModel.from_pretrained(model_name)

        # Define MLP head
        hidden_size = config['hidden_size']
        mlp_hidden_size = config['mlp_hidden_size']
        self.mlp = nn.Sequential(
            nn.Linear(2 * hidden_size, mlp_hidden_size),
            nn.ReLU(),
            nn.Linear(mlp_hidden_size, 1)  # Regression output
        )

    def forward(self, input1, attention_mask1, input2, attention_mask2):
        # Forward pass through BERT1
        outputs1 = self.bert1(input_ids=input1, attention_mask=attention_mask1).last_hidden_state
        print('shape of output1: ', outputs1.shape)
        print('shape of att mask1: ', attention_mask1.shape)
        cls1 = outputs1[:, 0, :]
        # cls1 = self.bert1(input_ids=input1, attention_mask=attention_mask1).last_hidden_state[:, 0, :]  # CLS token

        # Forward pass through BERT2
        outputs2 = self.bert2(input_ids=input2, attention_mask=attention_mask2).last_hidden_state  # CLS token
        print('shape of output2: ', outputs2.shape)
        print('shape of att mask2: ', attention_mask2.shape)
        cls2 = outputs2[:, 0, :]
        # cls2 = self.bert2(input_ids=input2, attention_mask=attention_mask2).last_hidden_state[:, 0, :]  # CLS token

        # Concatenate CLS embeddings
        combined_cls = torch.cat([cls1, cls2], dim=-1)  # Shape: [batch_size, 2 * hidden_size]

        # Pass through MLP head
        output = self.mlp(combined_cls)

        return output



##### Double-head BERT with [MASK] token-based regression and MLP

In [13]:
# two berts for two text features with MLP head for regression over MASK token embedding
class MASKPoolDualBERTWithMLP(nn.Module):
    def __init__(self, config):
        super(MASKPoolDualBERTWithMLP, self).__init__()
        model_name = config['model_name']
        # Initialize two independent BERT models
        self.bert1 = AutoModel.from_pretrained(model_name)
        self.bert2 = AutoModel.from_pretrained(model_name)

        # Define MLP head
        hidden_size = config['hidden_size']
        mlp_hidden_size = config['mlp_hidden_size']
        self.mask_token_index = config['mask_token_index']
        self.mlp = nn.Sequential(
            nn.Linear(3 * hidden_size, mlp_hidden_size),
            nn.ReLU(),
            nn.Linear(mlp_hidden_size, 1)  # Regression output
        )

    def forward(self, input1, attention_mask1, input2, attention_mask2):
        # Forward pass through BERT1
        output1 = self.bert1(input_ids=input1, attention_mask=attention_mask1).last_hidden_state
        cls1 = output1[:, 0, :]
        mask1 = output1[:, self.mask_token_index, :]

        # Forward pass through BERT2
        output2 = self.bert2(input_ids=input2, attention_mask=attention_mask2).last_hidden_state
        cls2 = output2[:, 0, :]

        # concat mask embeddings
        combined_mask = torch.cat([cls1, mask1, cls2], dim=-1)  # Shape: [batch_size, 3 * hidden_size]

        # Pass through MLP head
        output = self.mlp(combined_mask)
        return output


# one bert for two text features with MLP head for regression over MASK token embedding
class MASKPoolSingleBERTWithMLP(nn.Module):
    def __init__(self, config):
        super(MASKPoolSingleBERTWithMLP, self).__init__()
        model_name = config['model_name']
        # Initialize a BERT model
        self.bert = AutoModel.from_pretrained(model_name)

        # Define MLP head
        hidden_size = config['hidden_size']
        mlp_hidden_size = config['mlp_hidden_size']
        self.mask_token_index = config['mask_token_index']
        self.mlp = nn.Sequential(
            nn.Linear(3 * hidden_size, mlp_hidden_size),
            nn.ReLU(),
            nn.Linear(mlp_hidden_size, 1)  # Regression output
        )

    def forward(self, input1, attention_mask1, input2, attention_mask2):
        # Forward passes
        output1 = self.bert(input_ids=input1, attention_mask=attention_mask1).last_hidden_state
        cls1 = output1[:, 0, :]
        mask1 = output1[:, self.mask_token_index, :]

        output2 = self.bert(input_ids=input2, attention_mask=attention_mask2).last_hidden_state
        cls2 = output2[:, 0, :]

        # concat mask embeddings
        combined_mask = torch.cat([cls1, mask1, cls2], dim=-1)  # Shape: [batch_size, 3 * hidden_size]

        # Pass through MLP head
        output = self.mlp(combined_mask)
        return output

##### Double-head BERT with TSDAE pre-tuning and MLP

In [14]:
class TSDAEDualBERTWithMLP(nn.Module):
    """two berts for two text features with MLP head for regression.
    The model is pre-tuned with TSDAE."""
    def __init__(self, config, bert1, bert2):
        super(TSDAEDualBERTWithMLP, self).__init__()
        # Load TSDAE-ed BERT models
        self.bert1 = bert1
        self.bert2 = bert2

        # Define MLP head
        hidden_size = config['hidden_size']
        mlp_hidden_size = config['mlp_hidden_size']
        self.mlp = nn.Sequential(
            nn.Linear(2 * hidden_size, mlp_hidden_size),
            nn.ReLU(),
            nn.Linear(mlp_hidden_size, 1)  # Regression output
        )

    def forward(self, input1, attention_mask1, input2, attention_mask2):
        # idea from: https://github.com/UKPLab/sentence-transformers/issues/2494
        # Forward pass through BERT1
        input_dict1 = {
            'input_ids': input1,
            'attention_mask': attention_mask1
        }
        cls1 = self.bert1(input_dict1)['sentence_embedding']
        
        # Forward pass through BERT2
        input_dict2 = {
            'input_ids': input2,
            'attention_mask': attention_mask2
        }
        cls2 = self.bert2(input_dict2)['sentence_embedding']

        # Concatenate CLS embeddings
        combined_cls = torch.cat([cls1, cls2], dim=-1)  # Shape: [batch_size, 2 * hidden_size]

        # Pass through MLP head
        output = self.mlp(combined_cls)
        return output

##### Double-head BERT with multitask learning and MLP

In [15]:
# two berts for two text features with MLP head for regression
class MaskBinDualBERTWithMLP(nn.Module):
    """two berts for two text features with regression over CLS token
    and cosine similarity loss between MASK token and bin embeddings."""
    def __init__(self, config):
        super(MaskBinDualBERTWithMLP, self).__init__()

        # Initialize two independent BERT models
        model_name = config['model_name']
        self.bert1 = AutoModel.from_pretrained(model_name)
        self.bert2 = AutoModel.from_pretrained(model_name)

        # Add new bin tokens to the tokenizer and resize the model embeddings
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        new_tokens = ["[BIN_0]", "[BIN_1]", "[BIN_2]", "[BIN_3]", "[BIN_4]", "[BIN_5]", "[BIN_6]", "[BIN_7]",
                      "[BIN_8]", "[BIN_9]", "[BIN_10]", "[BIN_11]", "[BIN_12]", "[BIN_13]", "[BIN_14]"]
        self.tokenizer.add_tokens(new_tokens)
        self.bert1.resize_token_embeddings(len(self.tokenizer))  # Resize the model to accommodate new tokens
        self.mask_token_index = config['mask_token_index']

        # Define MLP head for regression
        hidden_size = config['hidden_size']
        mlp_hidden_size = config['mlp_hidden_size']
        self.mlp = nn.Sequential(
            nn.Linear(2 * hidden_size, mlp_hidden_size),
            nn.ReLU(),
            nn.Linear(mlp_hidden_size, 1)  # Regression output
        )

        # CosineEmbeddingLoss setup for MASK token prediction
        self.criterion_similarity = nn.CosineEmbeddingLoss(margin=0.0)


    def get_tokenizer(self):
        return self.tokenizer


    def forward(self, input1, attention_mask1, input2, attention_mask2, bin_ids=None):
        # Forward pass through BERT1 (for MASK token prediction and bin prediction)
        outputs1 = self.bert1(input_ids=input1, attention_mask=attention_mask1)
        cls1 = outputs1.last_hidden_state[:, 0, :]  # CLS token
        mask_idx = (input2 == self.mask_token_index).nonzero(as_tuple=True)  # Assume [MASK] token is at index self.mask_token_index
        mask_embedding = outputs1.last_hidden_state[mask_idx]  # Mask token embedding

        # Forward pass through BERT2 (for CLS token regression)
        cls2 = self.bert2(input_ids=input2, attention_mask=attention_mask2).last_hidden_state[:, 0, :]  # CLS token
        
        # Concatenate CLS tokens from BERT1 and BERT2 and pass through MLP for regression output
        combined_cls = torch.cat([cls1, cls2], dim=-1)  # Concatenate CLS embeddings
        
        # Pass through MLP head for salary regression
        salary_output = self.mlp(combined_cls)
        
        # If bin_ids are provided, compute cosine similarity between MASK token and bin embeddings
        if bin_ids is not None:
            # Retrieve bin embeddings from the BERT model's embedding layer
            print(bin_ids)
            bin_embedding = self.bert1.embeddings.word_embeddings(bin_ids)  # [batch_size, hidden_size]
            print(f"mask_embedding shape: {mask_embedding.shape}")
            print(f"bin_embedding shape: {bin_embedding.shape}")
            similarity_loss = self.criterion_similarity(mask_embedding, bin_embedding, torch.ones(mask_embedding.size(0)))
            return salary_output, similarity_loss
        
        return salary_output, None  # For inference, return salary output only

#### Training methods

##### Baseline regression

In [16]:
from torch.utils.data import DataLoader
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error


def fit_eval(
    seed,
    model,
    X_train,
    X_test,
    y_train_reg,
    y_test_reg,
    criterion,
    tokenizer,
    config,
    text_col_1,
    text_col_2,
):
    set_seed(seed)
    
    # Memory cleanup
    memory_cleanup()

    # Unpack config
    learning_rate = config["learning_rate"]
    num_epochs = config["num_epochs"]
    batch_size = config["batch_size"]
    if "seq_length" in config:
        seq_length1 = config["seq_length"]
        seq_length2 = config["seq_length"]
    else:
        seq_length1 = config["seq_length_1"]
        seq_length2 = config["seq_length_2"]

    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Make datasets
    train_dataset = DualTextDataset(X_train, text_col_1, text_col_2, y_train_reg, tokenizer, seq_length1, seq_length2)
    test_dataset = DualTextDataset(X_test, text_col_1, text_col_2, y_test_reg, tokenizer, seq_length1, seq_length2)
    # Make dataloaders
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Training and Evaluation Loop
    history = {
                "train_loss": [],
               "test_loss": [], 
                "train_rmse": [],
               "test_rmse": [], 
               "train_r2": [],
               "train_r2": [],
               "test_r2": [],
               "train_mae": [],
               "test_mae": [],
               "y_pred": [],
               "y_test": [],
               }

    print('Starting training/eval loop...')
    for epoch in range(num_epochs):
        print('Starting training...')
        # Training Phase
        model.train()
        train_losses = []
        all_preds = []
        all_labels = []
        for batch in train_dataloader:
            inputs1, inputs2, targets = batch
            input1 = inputs1["input_ids"].squeeze(1).to(device)
            attention_mask1 = inputs1["attention_mask"].squeeze(1).to(device)
            input2 = inputs2["input_ids"].squeeze(1).to(device)
            attention_mask2 = inputs2["attention_mask"].squeeze(1).to(device)
            targets = targets.to(device)
    
            optimizer.zero_grad()
            outputs = model(input1, attention_mask1, input2, attention_mask2)
            outputs = outputs.flatten()

            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            train_losses.append(loss.item())

            all_preds.extend(outputs.cpu().detach().numpy())
            all_labels.extend(targets.cpu().numpy())
    
        train_loss = np.mean(train_losses)
        
        train_r2 = r2_score(all_labels, all_preds)

        train_rmse = mean_squared_error(all_labels, all_preds, squared=False)
        
        train_mae = mean_absolute_error(all_labels, all_preds)
        
        history["train_loss"].append(train_loss)
        history["train_r2"].append(train_r2)
        history["train_rmse"].append(train_rmse)
        history["train_mae"].append(train_mae)
    
        # Evaluation Phase
        print('Epoch done, evaluating...')
        model.eval()
        test_losses = []
        all_preds = []
        all_labels = []
        with torch.no_grad():
            for batch in test_dataloader:
                inputs1, inputs2, targets = batch
                input1 = inputs1["input_ids"].squeeze(1).to(device)
                attention_mask1 = inputs1["attention_mask"].squeeze(1).to(device)
                input2 = inputs2["input_ids"].squeeze(1).to(device)
                attention_mask2 = inputs2["attention_mask"].squeeze(1).to(device)
                targets = targets.to(device)

                outputs = model(input1, attention_mask1, input2, attention_mask2)
                outputs = outputs.flatten()

                loss = criterion(outputs, targets)
                test_losses.append(loss.item())

                all_preds.extend(outputs.cpu().numpy())
                all_labels.extend(targets.cpu().numpy())

        history["y_pred"].append(all_preds)
        history["y_test"].append(all_labels)

        test_loss = np.mean(test_losses)

        test_r2 = r2_score(all_labels, all_preds)
        
        test_rmse = mean_squared_error(all_labels, all_preds, squared=False)
        
        test_mae = mean_absolute_error(all_labels, all_preds)
    
        history["test_loss"].append(test_loss)
        
        history["test_r2"].append(test_r2)
        
        history["test_rmse"].append(test_rmse)
        
        history["test_mae"].append(test_mae)
    
        print(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, "
              f"Train R2: {train_r2:.4f}, Test R2: {test_r2:.4f}")

        print(f"Epoch {epoch + 1}/{num_epochs}, Train MAE: {train_mae:.4f}, Test MAE: {test_mae:.4f}")

    return model, history

##### Single bert, cross attention

In [17]:
class CrossAttentionLayer(nn.Module):
    def __init__(self, hidden_size, num_heads=8): 
        super(CrossAttentionLayer, self).__init__()
        self.attention = nn.MultiheadAttention(
            embed_dim=hidden_size,
            num_heads=num_heads,
            batch_first=True)
        self.layer_norm = nn.LayerNorm(hidden_size)


    def forward(self, query, key, value, attn_mask=None, key_padding_mask=None):
        # Apply multi-head attention
        attn_output, _ = self.attention(query, key, value, attn_mask=attn_mask, key_padding_mask=key_padding_mask)
        # nan_count = torch.isnan(attn_output).sum().item()
        # print('nan count after attention,', nan_count)
        
        # Add residual connection and layer norm
        output = self.layer_norm(query + attn_output)  # residual connection with query
        return output


class SingleBERTWithCrossAttention(nn.Module):
    """Single BERT model with cross-attention between the two text features.
    Here, query is the first text feature and key, value are the second text feature."""
    def __init__(self, config):
        super(SingleBERTWithCrossAttention, self).__init__()
        model_name = config['model_name']
        self.bert = AutoModel.from_pretrained(model_name)

        # Define MLP head
        hidden_size = config['hidden_size']
        mlp_hidden_size = config['mlp_hidden_size']
        self.mlp = nn.Sequential(
            nn.Linear(2 * hidden_size, mlp_hidden_size),
            nn.ReLU(),
            nn.Linear(mlp_hidden_size, 1)  # Regression output
        )

        self.num_heads = config['num_heads']
        self.cross_attention = CrossAttentionLayer(hidden_size, num_heads=self.num_heads)
      
    def forward(self, input1: torch.Tensor, attention_mask1: torch.Tensor, input2: torch.Tensor, attention_mask2: torch.Tensor):
        # Get BERT outputs
        outputs1 = self.bert(input_ids=input1, attention_mask=attention_mask1).last_hidden_state
        outputs2 = self.bert(input_ids=input2, attention_mask=attention_mask2).last_hidden_state
        
        # Get raw CLS token before attention
        cls2 = outputs2[:, 0, :]

        # prepare key_padding_mask
        key_padding_mask = (attention_mask2 == 0).bool() # True indicates positions to exclude (padding tokens). Shape: (batch_size, source_len)

        # prepare attention mask
        # # Step 1: Expand to match query and key dimensions
        # original shape: (batch_size, target_len)
        attn_mask = attention_mask1.unsqueeze(2) # create a new dimension at the end to be able to expand. Shape: (batch_size, target_len, 1)
        attn_mask = attn_mask.expand(-1, -1, attention_mask2.size(1)) # expand to match the source_len. Shape: (batch_size, target_len, source_len)

        # # Step 2: Adjust for multi-head attention
        attn_mask = attn_mask.unsqueeze(1) # Add head dimension at position 1. Shape: (batch_size, 1, target_len, source_len)
        attn_mask = attn_mask.repeat(1, self.num_heads, 1, 1)  # Repeat for each head. Shape: (batch_size, num_heads, target_len, source_len)
        attn_mask = attn_mask.view(-1, attention_mask1.size(1), attention_mask2.size(1))  # Merge batch and head dimensions. Shape: (batch_size * num_heads, target_len, source_len)

        # # Step 3: Convert to boolean mask
        attn_mask = (attn_mask == 0).bool()  # True indicates positions to exclude (padding tokens). Shape: (batch_size * num_heads, target_len, source_len)

        # Apply cross-attention
        attended_features = self.cross_attention(
            query=outputs1,
            key=outputs2,
            value=outputs2,
            key_padding_mask=key_padding_mask,
            # attn_mask=attn_mask,
        )

        # nan_count = torch.isnan(attended_features).sum().item()
        # print('att features shape,', attended_features.shape)
        # print('nan_count,', nan_count)

        # Get CLS from attended features
        cls1_attended = attended_features[:, 0, :]

        # Concatenate the two [CLS] tokens
        combined = torch.cat([cls1_attended, cls2], dim=1) # Shape: (batch_size, 2 * hidden_size)

        # Pass through MLP head
        output = self.mlp(combined)

        return output

### Training-eval loop with experiments

#### Data preprocessing

##### Define text feature/target columns

In [18]:
text_col_1 = 'description_no_numbers'
text_col_1_with_prompt = text_col_1 + '_with_prompt' # Add prompt to text for multitask learning

text_col_2 = 'title_company_location_skills_source' # Merged text column, second feature

bin_targets_col = 'salary_bin' # Bin targets for MASK token prediction and multitask learning
target_col = 'log_salary_from' # regression target

##### Create merged title/skills/location/source feature

In [19]:
df['skills'] = df['skills'].fillna('Не указаны')

title_company_location_skills_feature_template = """
Позиция: {position}
Компания: {company}
Место: {location}
Навыки: {skills}
Источник: {source}
"""

df[text_col_2] = df.apply(lambda x: title_company_location_skills_feature_template.format(
    position=x['title'],
    company=x['company'],
    location=x['location'],
    skills=x['skills'],
    source=x['source']
), axis=1)

##### Create bins for target prediction

In [20]:
# prompt to be added to feature 1 for multitask learning
prompt = """\
[CLS] Далее указано описание вакансии. \
Судя по описанию, зарплата на этой позиции составляет [MASK].[SEP]\
"""

df[text_col_1_with_prompt] = prompt + df[text_col_1]

#### Training code

In [21]:
memory_cleanup()

##### Experiment 1: Single BERT, MSE loss

In [22]:
# import torch.nn as nn
# import pickle
# import warnings
# from transformers import AutoTokenizer
# from sklearn.model_selection import train_test_split


# experiment_name = 'single_bert_mse'
# print(experiment_name.upper())
# print('='*100)
# print()

# # Suppress all FutureWarnings
# warnings.simplefilter(action='ignore', category=FutureWarning)

# seeds = [42, 78687, 123123]
# combined_history = {}

# # Hyperparameters and configuration
# config = {
#     "model_name": "sergeyzh/rubert-tiny-turbo",
#     "batch_size": 32,
#     "seq_length": 1024,
#     "hidden_size": 312,
#     "mlp_hidden_size": 128,
#     "num_epochs": 10,
#     "learning_rate": 5e-6,
#     "mask_token_index": 17, # position of the [MASK] token in the feature 1 (to be used in MASK token prediction)
# }

# memory_cleanup()

# model_name = config['model_name']
# # Tokenizer
# tokenizer = AutoTokenizer.from_pretrained(model_name)

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Dataset and DataLoader
# # Prepare data
# X = df[[text_col_1, text_col_1_with_prompt, text_col_2]]
# y = df[[target_col, bin_targets_col]]

# for seed in seeds:
#     memory_cleanup()
#     print()
#     print(f'Starting for seed {str(seed)}...')
#     print('-' * 100)
#     print()

#     combined_history[seed] = {}

#     set_seed(seed)

#     # Split train-test
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)
#     y_train_reg = y_train[target_col]
#     y_test_reg = y_test[target_col]

#     # Initialize the model
#     model = SingleBERTWithMLP(config)
#     model = torch.nn.DataParallel(model).to(device)

#     # Loss Function
#     criterion = nn.MSELoss()

#     model, history = fit_eval(
#         seed,
#         model,
#         X_train,
#         X_test,
#         y_train_reg,
#         y_test_reg,
#         criterion,
#         tokenizer,
#         config,
#         text_col_1,
#         text_col_2,
#     )

#     memory_cleanup()

#     combined_history[seed] = history    

# # Display metrics
# display_metrics_with_ci(combined_history)

# # save the history as pickle
# with open(f'./data/history/transfomers_{experiment_name}.pickle', 'wb') as handle:
#     pickle.dump(combined_history, handle, protocol=pickle.HIGHEST_PROTOCOL)

##### Experiment 2: Dual BERT, MSE loss

In [23]:
# import torch.nn as nn
# import pickle
# import warnings
# from transformers import AutoTokenizer
# from sklearn.model_selection import train_test_split


# experiment_name = 'dual_bert_mse'
# print(experiment_name.upper())
# print('='*100)
# print()

# # Suppress all FutureWarnings
# warnings.simplefilter(action='ignore', category=FutureWarning)

# seeds = [42, 78687, 123123]
# combined_history = {}

# # Hyperparameters and configuration
# config = {
#     "model_name": "sergeyzh/rubert-tiny-turbo",
#     "batch_size": 32,
#     "seq_length": 1024,
#     "hidden_size": 312,
#     "mlp_hidden_size": 128,
#     "num_epochs": 10,
#     "learning_rate": 5e-6,
#     "mask_token_index": 17, # position of the [MASK] token in the feature 1 (to be used in MASK token prediction)
# }

# memory_cleanup()

# model_name = config['model_name']
# # Tokenizer
# tokenizer = AutoTokenizer.from_pretrained(model_name)

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Dataset and DataLoader
# # Prepare data
# X = df[[text_col_1, text_col_1_with_prompt, text_col_2]][:200]
# y = df[[target_col,]][:200]


# for seed in seeds:
#     memory_cleanup()
#     print()
#     print(f'Starting for seed {str(seed)}...')
#     print('-' * 100)
#     print()

#     combined_history[seed] = {}

#     set_seed(seed)

#     # Split train-test
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)
#     y_train_reg = y_train[target_col]
#     y_test_reg = y_test[target_col]

#     # Initialize the model
#     model = DualBERTWithMLP(config)
#     model = torch.nn.DataParallel(model).to(device)

#     # Loss Function
#     criterion = nn.MSELoss()

#     model, history = fit_eval(
#         seed,
#         model,
#         X_train,
#         X_test,
#         y_train_reg,
#         y_test_reg,
#         criterion,
#         tokenizer,
#         config,
#         text_col_1,
#         text_col_2,
#     )

#     memory_cleanup()

#     combined_history[seed] = history    

# # Display metrics
# display_metrics_with_ci(combined_history)

# # save the history as pickle
# with open(f'./data/history/transfomers_{experiment_name}.pickle', 'wb') as handle:
#     pickle.dump(combined_history, handle, protocol=pickle.HIGHEST_PROTOCOL)


# # shape of output1:  torch.Size([16, 1024, 312])
# # shape of output2:  torch.Size([16, 256, 312])

# # shape of att mask1:  torch.Size([16, 1024])
# # shape of att mask2:  torch.Size([16, 256])

# # attn_mask (Optional[Tensor]) – If specified, a 2D or 3D mask preventing attention to certain positions. 
# # Must be of shape (L,S) or (N⋅num_heads,L,S),
# # where NN is the batch size, L is the target sequence length, and S is the source sequence length. 
# # A 2D mask will be broadcasted across the batch while a 3D mask allows for a different mask for each entry in the batch. 
# # Binary and float masks are supported. 
# # For a binary mask, a True value indicates that the corresponding position is not allowed to attend.
# # For a float mask, the mask values will be added to the attention weight.
# # If both attn_mask and key_padding_mask are supplied, their types should match.

# # shape: 32 * 8, 1024, 256

##### Experiment 3: Single BERT, Huber loss

In [24]:
# import torch.nn as nn
# import pickle
# import warnings
# from transformers import AutoTokenizer
# from sklearn.model_selection import train_test_split


# experiment_name = 'single_bert_huber'
# print(experiment_name.upper())
# print('='*100)
# print()

# # Suppress all FutureWarnings
# warnings.simplefilter(action='ignore', category=FutureWarning)

# seeds = [42, 78687, 123123]
# combined_history = {}

# # Hyperparameters and configuration
# config = {
#     "model_name": "sergeyzh/rubert-tiny-turbo",
#     "batch_size": 32,
#     "seq_length": 1024,
#     "hidden_size": 312,
#     "mlp_hidden_size": 128,
#     "num_epochs": 10,
#     "learning_rate": 5e-6,
#     "mask_token_index": 17, # position of the [MASK] token in the feature 1 (to be used in MASK token prediction)
# }

# memory_cleanup()

# model_name = config['model_name']
# # Tokenizer
# tokenizer = AutoTokenizer.from_pretrained(model_name)

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Dataset and DataLoader
# # Prepare data
# X = df[[text_col_1, text_col_1_with_prompt, text_col_2]]
# y = df[[target_col, bin_targets_col]]

# for seed in seeds:
#     memory_cleanup()
#     print()
#     print(f'Starting for seed {str(seed)}...')
#     print('-' * 100)
#     print()

#     combined_history[seed] = {}

#     set_seed(seed)

#     # Split train-test
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)
#     y_train_reg = y_train[target_col]
#     y_test_reg = y_test[target_col]

#     # Initialize the model
#     model = SingleBERTWithMLP(config)
#     model = torch.nn.DataParallel(model).to(device)

#     # Loss Function
#     criterion = nn.HuberLoss() # changed!

#     model, history = fit_eval(
#         seed,
#         model,
#         X_train,
#         X_test,
#         y_train_reg,
#         y_test_reg,
#         criterion,
#         tokenizer,
#         config,
#         text_col_1,
#         text_col_2,
#     )

#     memory_cleanup()

#     combined_history[seed] = history    

# # Display metrics
# display_metrics_with_ci(combined_history)

# # save the history as pickle
# with open(f'./data/history/transfomers_{experiment_name}.pickle', 'wb') as handle:
#     pickle.dump(combined_history, handle, protocol=pickle.HIGHEST_PROTOCOL)

##### Experiment 4: Double BERT, Huber loss: pre-tune with TSDAE

In [25]:
# import torch.nn as nn
# import pickle
# import warnings
# from transformers import AutoTokenizer
# from sklearn.model_selection import train_test_split


# experiment_name = 'double_bert_huber_tsdae'
# print(experiment_name.upper())
# print('='*100)
# print()

# # Suppress all FutureWarnings
# warnings.simplefilter(action='ignore', category=FutureWarning)

# seeds = [42, 78687, 123123]
# combined_history = {}

# # Hyperparameters and configuration
# config = {
#     "model_name": "sergeyzh/rubert-tiny-turbo",
#     "batch_size": 32,
#     "seq_length": 1024,
#     "hidden_size": 312,
#     "mlp_hidden_size": 128,
#     "num_epochs": 10,
#     "learning_rate": 5e-6,
#     "mask_token_index": 17, # position of the [MASK] token in the feature 1 (to be used in MASK token prediction)
# }

# memory_cleanup()

# model_name = config['model_name']
# # Tokenizer
# tokenizer = AutoTokenizer.from_pretrained(model_name)

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Dataset and DataLoader
# # Prepare data
# X = df[[text_col_1, text_col_1_with_prompt, text_col_2]]
# y = df[[target_col, bin_targets_col]]

# for seed in seeds:
#     memory_cleanup()
#     print()
#     print(f'Starting for seed {str(seed)}...')
#     print('-' * 100)
#     print()

#     combined_history[seed] = {}

#     set_seed(seed)

#     # Split train-test
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)
#     # further split train data into regression train and tsdae train data
#     X_train_tsdae, X_tsdae, y_train_tsdae, _ = train_test_split(X_train, y_train, test_size=0.01, random_state=seed)

#     # get test and train targets
#     y_test_reg = y_test[target_col]
#     y_train_tsdae_reg = y_train_tsdae[target_col]

#     # X_tsdae used for TSDAE
#     # X_train_tsdae, y_train_tsdae used for the downstream regression training
#     # X_test, y_test used for the final model evaluation
#     # y_test_tsdae is not used (hence set to '_')

#     # TSDAE part:

#     # 1. convert job descriptions into a set of sentences
#     # and select 10-60 word sentences as a feature column.
#     # 2. Train with TSDAE task on these selected sentences.
    
#     # 1. Create a DataFrame of unique sentences and their lengths for X_tsdae
#     unique_sentences = []
#     unique_sentence_lengths = []
#     for text in X_tsdae[text_col_1]:
#         sentences, sentence_lengths = get_sentence_lengths(text)
#         unique_sentences.extend(sentences)
#         unique_sentence_lengths.extend(sentence_lengths)

#     unique_sentences_df = pd.DataFrame({
#         'sentence': unique_sentences,
#         'length': unique_sentence_lengths
#     })

#     unique_sentences = unique_sentences_df[(unique_sentences_df.length >= 10) & (unique_sentences_df.length <= 60)]['sentence']

#     # get array with features for each bert
#     train_sentences_array = [
#         unique_sentences.tolist(),
#         X_tsdae[text_col_2].tolist(),
#     ]

#     # 2. Train the models on TSDAE task
#     berts_after_tsdae = []
#     for index, train_sentences in enumerate(train_sentences_array):
#         memory_cleanup()
#         berts_after_tsdae.append(train_tsdae_bert(model_name, train_sentences))
#     memory_cleanup()

#     tsdae_bert1, tsdae_bert2 = berts_after_tsdae

#     # Initialize the model for regression
#     model = TSDAEDualBERTWithMLP(config, tsdae_bert1, tsdae_bert2)
#     model = torch.nn.DataParallel(model).to(device)

#     # Loss Function
#     criterion = nn.HuberLoss()

#     # fit-eval regression
#     model, history = fit_eval(
#         seed,
#         model,
#         X_train_tsdae,
#         X_test,
#         y_train_tsdae_reg,
#         y_test_reg,
#         criterion,
#         tokenizer,
#         config,
#         text_col_1,
#         text_col_2,
#     )

#     memory_cleanup()
    
#     combined_history[seed] = history

# # Display metrics
# display_metrics_with_ci(combined_history)

# # save the history as pickle
# with open(f'./data/history/transfomers_{experiment_name}.pickle', 'wb') as handle:
#     pickle.dump(combined_history, handle, protocol=pickle.HIGHEST_PROTOCOL)

#### Experiment 5, dual bert (cls + mask pooling), huber loss

In [26]:
# import torch.nn as nn
# import pickle
# import warnings
# from transformers import AutoTokenizer
# from sklearn.model_selection import train_test_split


# experiment_name = 'dual_bert_cls_mask_huber'
# print(experiment_name.upper())
# print('='*100)
# print()

# # Suppress all FutureWarnings
# warnings.simplefilter(action='ignore', category=FutureWarning)

# seeds = [42, 78687, 123123]
# combined_history = {}

# # Hyperparameters and configuration
# config = {
#     "model_name": "sergeyzh/rubert-tiny-turbo",
#     "batch_size": 32,
#     "seq_length": 1024,
#     "hidden_size": 312,
#     "mlp_hidden_size": 256,
#     "num_epochs": 10,
#     "learning_rate": 5e-6,
#     "mask_token_index": 15, # position of the [MASK] token in the feature 1 (to be used in MASK token prediction)
# }

# memory_cleanup()

# model_name = config['model_name']
# # Tokenizer
# tokenizer = AutoTokenizer.from_pretrained(model_name)

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Dataset and DataLoader
# # Prepare data
# X = df[[text_col_1, text_col_1_with_prompt, text_col_2]]
# y = df[[target_col]]


# for seed in seeds:
#     memory_cleanup()
#     print()
#     print(f'Starting for seed {str(seed)}...')
#     print('-' * 100)
#     print()

#     combined_history[seed] = {}

#     set_seed(seed)

#     # Split train-test
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)
#     y_train_reg = y_train[target_col]
#     y_test_reg = y_test[target_col]

#     # Initialize the model
#     model = MASKPoolDualBERTWithMLP(config)
#     model = torch.nn.DataParallel(model).to(device)

#     # Loss Function
#     criterion = nn.HuberLoss()

#     model, history = fit_eval(
#         seed,
#         model,
#         X_train,
#         X_test,
#         y_train_reg,
#         y_test_reg,
#         criterion,
#         tokenizer,
#         config,
#         text_col_1_with_prompt,
#         text_col_2,
#     )

#     memory_cleanup()

#     combined_history[seed] = history    

# # Display metrics
# display_metrics_with_ci(combined_history)

# # save the history as pickle
# with open(f'./data/history/transfomers_{experiment_name}.pickle', 'wb') as handle:
#     pickle.dump(combined_history, handle, protocol=pickle.HIGHEST_PROTOCOL)

#### Experiment 6, single bert + cross attention + huber loss

In [27]:
# import torch.nn as nn
# import pickle
# import warnings
# from transformers import AutoTokenizer
# from sklearn.model_selection import train_test_split
# import time


# experiment_name = 'single_bert_cross_attention_huber'
# print(experiment_name.upper())
# print('='*100)
# print()

# # Suppress all FutureWarnings
# warnings.simplefilter(action='ignore', category=FutureWarning)

# seeds = [42, 78687, 123123]
# combined_history = {}

# # Hyperparameters and configuration
# config = {
#     "model_name": "sergeyzh/rubert-tiny-turbo",
#     "batch_size": 32,
#     "seq_length_1": 1024,
#     "seq_length_2": 256,
#     "hidden_size": 312,
#     "mlp_hidden_size": 128,
#     "num_epochs": 10,
#     "learning_rate": 5e-6,
#     "mask_token_index": 17, # position of the [MASK] token in the feature 1 (to be used in MASK token prediction)
#     "num_heads": 8
# }

# memory_cleanup()

# model_name = config['model_name']
# # Tokenizer
# tokenizer = AutoTokenizer.from_pretrained(model_name)

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Dataset and DataLoader
# # Prepare data
# # short version
# # df_sample = df.sample(frac=1, random_state=42).reset_index(drop=True)
# # df_sample = df_sample.sample(n=5000, random_state=42)
# # X = df_sample[[text_col_1, text_col_1_with_prompt, text_col_2]]
# # y = df_sample[[target_col,]]

# # full version
# X = df[[text_col_1, text_col_1_with_prompt, text_col_2]]
# y = df[[target_col,]]

# start_time = time.time()

# for seed in seeds:
#     memory_cleanup()
#     print()
#     print(f'Starting for seed {str(seed)}...')
#     print('-' * 100)
#     print()

#     combined_history[seed] = {}

#     set_seed(seed)

#     # Split train-test
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)
#     y_train_reg = y_train[target_col]
#     y_test_reg = y_test[target_col]

#     # Initialize the model
#     model = SingleBERTWithCrossAttention(config)
#     model = torch.nn.DataParallel(model).to(device)

#     # Loss Function
#     criterion = nn.HuberLoss() # changed!

#     model, history = fit_eval(
#         seed,
#         model,
#         X_train,
#         X_test,
#         y_train_reg,
#         y_test_reg,
#         criterion,
#         tokenizer,
#         config,
#         text_col_1,
#         text_col_2,
#     )

#     memory_cleanup()

#     combined_history[seed] = history    

# # Display metrics
# display_metrics_with_ci(combined_history)

# # save the history as pickle
# with open(f'./data/history/transfomers_{experiment_name}.pickle', 'wb') as handle:
#     pickle.dump(combined_history, handle, protocol=pickle.HIGHEST_PROTOCOL)

# end_time = time.time()
# execution_time = end_time - start_time
# print(f"Execution time: {execution_time:.2f} seconds")

# # no attention:
# # 1694.4s	178	TEST METRICS FOR THE BEST EPOCH (#10)
# # 1694.4s	179	R2: mean = 0.4464, 95% CI = [0.3873, 0.5055]
# # 1694.4s	180	MAE: mean = 0.3759, 95% CI = [0.3352, 0.4166]
# # 1694.4s	181	RMSE: mean = 0.4848, 95% CI = [0.4521, 0.5175]

##### 8. single bert huber short seq 2

In [28]:
# import torch.nn as nn
# import pickle
# import warnings
# from transformers import AutoTokenizer
# from sklearn.model_selection import train_test_split
# import time


# experiment_name = 'single_bert_huber_short_seq_len2'
# print(experiment_name.upper())
# print('='*100)
# print()

# # Suppress all FutureWarnings
# warnings.simplefilter(action='ignore', category=FutureWarning)

# seeds = [42, 78687, 123123]
# combined_history = {}

# # Hyperparameters and configuration
# config = {
#     "model_name": "sergeyzh/rubert-tiny-turbo",
#     "batch_size": 32,
#     "seq_length_1": 1024,
#     "seq_length_2": 256,
#     "hidden_size": 312,
#     "mlp_hidden_size": 128,
#     "num_epochs": 10,
#     "learning_rate": 5e-6,
#     "mask_token_index": 17, # position of the [MASK] token in the feature 1 (to be used in MASK token prediction)
#     "num_heads": 8
# }

# memory_cleanup()

# model_name = config['model_name']
# # Tokenizer
# tokenizer = AutoTokenizer.from_pretrained(model_name)

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Dataset and DataLoader
# # Prepare data
# # short version
# # df_sample = df.sample(frac=1, random_state=42).reset_index(drop=True)
# # df_sample = df_sample.sample(n=5000, random_state=42)
# # X = df_sample[[text_col_1, text_col_1_with_prompt, text_col_2]]
# # y = df_sample[[target_col,]]

# # full version
# X = df[[text_col_1, text_col_1_with_prompt, text_col_2]]
# y = df[[target_col,]]

# start_time = time.time()

# for seed in seeds:
#     memory_cleanup()
#     print()
#     print(f'Starting for seed {str(seed)}...')
#     print('-' * 100)
#     print()

#     combined_history[seed] = {}

#     set_seed(seed)

#     # Split train-test
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)
#     y_train_reg = y_train[target_col]
#     y_test_reg = y_test[target_col]

#     # Initialize the model
#     model = SingleBERTWithMLP(config)
#     model = torch.nn.DataParallel(model).to(device)

#     # Loss Function
#     criterion = nn.HuberLoss() # changed!

#     model, history = fit_eval(
#         seed,
#         model,
#         X_train,
#         X_test,
#         y_train_reg,
#         y_test_reg,
#         criterion,
#         tokenizer,
#         config,
#         text_col_1,
#         text_col_2,
#     )

#     memory_cleanup()

#     combined_history[seed] = history    

# # Display metrics
# display_metrics_with_ci(combined_history)

# # save the history as pickle
# with open(f'./data/history/transfomers_{experiment_name}.pickle', 'wb') as handle:
#     pickle.dump(combined_history, handle, protocol=pickle.HIGHEST_PROTOCOL)

# end_time = time.time()
# execution_time = end_time - start_time
# print(f"Execution time: {execution_time:.2f} seconds")

# # no attention:
# # 1694.4s	178	TEST METRICS FOR THE BEST EPOCH (#10)
# # 1694.4s	179	R2: mean = 0.4464, 95% CI = [0.3873, 0.5055]
# # 1694.4s	180	MAE: mean = 0.3759, 95% CI = [0.3352, 0.4166]
# # 1694.4s	181	RMSE: mean = 0.4848, 95% CI = [0.4521, 0.5175]

#### Experiment 8, single bert + huber loss + mask pooling

In [29]:
# import torch.nn as nn
# import pickle
# import warnings
# from transformers import AutoTokenizer
# from sklearn.model_selection import train_test_split
# import time


# experiment_name = 'single_bert_huber_mask_pooling'
# print(experiment_name.upper())
# print('='*100)
# print()

# # Suppress all FutureWarnings
# warnings.simplefilter(action='ignore', category=FutureWarning)

# seeds = [42, 78687, 123123]
# combined_history = {}

# # Hyperparameters and configuration
# config = {
#     "model_name": "sergeyzh/rubert-tiny-turbo",
#     "batch_size": 32,
#     "seq_length_1": 1024,
#     "seq_length_2": 1024,
#     "hidden_size": 312,
#     "mlp_hidden_size": 128,
#     "num_epochs": 10,
#     "learning_rate": 5e-6,
#     "mask_token_index": 17, # position of the [MASK] token in the feature 1 (to be used in MASK token prediction)
#     "num_heads": 8
# }

# memory_cleanup()

# model_name = config['model_name']
# # Tokenizer
# tokenizer = AutoTokenizer.from_pretrained(model_name)

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Dataset and DataLoader
# # Prepare data
# # short version
# # df_sample = df.sample(frac=1, random_state=42).reset_index(drop=True)
# # df_sample = df_sample.sample(n=5000, random_state=42)
# # X = df_sample[[text_col_1, text_col_1_with_prompt, text_col_2]]
# # y = df_sample[[target_col,]]

# # full version
# X = df[[text_col_1, text_col_1_with_prompt, text_col_2]]
# y = df[[target_col,]]

# start_time = time.time()

# for seed in seeds:
#     memory_cleanup()
#     print()
#     print(f'Starting for seed {str(seed)}...')
#     print('-' * 100)
#     print()

#     combined_history[seed] = {}

#     set_seed(seed)

#     # Split train-test
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)
#     y_train_reg = y_train[target_col]
#     y_test_reg = y_test[target_col]

#     # Initialize the model
#     model = MASKPoolSingleBERTWithMLP(config)
#     model = torch.nn.DataParallel(model).to(device)

#     # Loss Function
#     criterion = nn.HuberLoss() # changed!

#     model, history = fit_eval(
#         seed,
#         model,
#         X_train,
#         X_test,
#         y_train_reg,
#         y_test_reg,
#         criterion,
#         tokenizer,
#         config,
#         text_col_1,
#         text_col_2,
#     )

#     memory_cleanup()

#     combined_history[seed] = history    

# # Display metrics
# display_metrics_with_ci(combined_history)

# # save the history as pickle
# with open(f'./data/history/transfomers_{experiment_name}.pickle', 'wb') as handle:
#     pickle.dump(combined_history, handle, protocol=pickle.HIGHEST_PROTOCOL)

# end_time = time.time()
# execution_time = end_time - start_time
# print(f"Execution time: {execution_time:.2f} seconds")

# # no attention:
# # 1694.4s	178	TEST METRICS FOR THE BEST EPOCH (#10)
# # 1694.4s	179	R2: mean = 0.4464, 95% CI = [0.3873, 0.5055]
# # 1694.4s	180	MAE: mean = 0.3759, 95% CI = [0.3352, 0.4166]
# # 1694.4s	181	RMSE: mean = 0.4848, 95% CI = [0.4521, 0.5175]

#### Experiment 9 (baseline), average predictions

In [32]:
# predict by average
import torch.nn as nn
import pickle
import warnings
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split
import time
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


experiment_name = 'average'
print(experiment_name.upper())
print('='*100)
print()

# Suppress all FutureWarnings
warnings.simplefilter(action='ignore', category=FutureWarning)

seeds = [42, 78687, 123123]
combined_history = {}

# Hyperparameters and configuration
config = {
    "model_name": "sergeyzh/rubert-tiny-turbo",
    "batch_size": 32,
    "seq_length_1": 1024,
    "seq_length_2": 1024,
    "hidden_size": 312,
    "mlp_hidden_size": 128,
    "num_epochs": 10,
    "learning_rate": 5e-6,
    "mask_token_index": 17, # position of the [MASK] token in the feature 1 (to be used in MASK token prediction)
    "num_heads": 8
}

memory_cleanup()

model_name = config['model_name']
# Tokenizer
# tokenizer = AutoTokenizer.from_pretrained(model_name)

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Dataset and DataLoader
# Prepare data
# short version
# df_sample = df.sample(frac=1, random_state=42).reset_index(drop=True)
# df_sample = df_sample.sample(n=5000, random_state=42)
# X = df_sample[[text_col_1, text_col_1_with_prompt, text_col_2]]
# y = df_sample[[target_col,]]

# full version
X = df[[text_col_1, text_col_1_with_prompt, text_col_2]]
y = df[[target_col,]]

start_time = time.time()

for seed in seeds:
    memory_cleanup()
    print()
    print(f'Starting for seed {str(seed)}...')
    print('-' * 100)
    print()

    combined_history[seed] = {}

    set_seed(seed)

    # Split train-test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)
    y_train_reg = y_train[target_col]
    y_test_reg = y_test[target_col]

    # Initialize the model
    # model = MASKPoolSingleBERTWithMLP(config)
    # model = torch.nn.DataParallel(model).to(device)

    # Loss Function
    # criterion = nn.HuberLoss() # changed!

    # model, history = fit_eval(
    #     seed,
    #     model,
    #     X_train,
    #     X_test,
    #     y_train_reg,
    #     y_test_reg,
    #     criterion,
    #     tokenizer,
    #     config,
    #     text_col_1,
    #     text_col_2,
    # )

    y_pred = y_train_reg.mean().repeat(len(y_test_reg))
    y_train_by_average = y_train_reg.mean().repeat(len(y_train_reg))
    train_r2 = r2_score(y_train_reg, y_train_by_average)
    train_mae = mean_absolute_error(y_train_reg, y_train_by_average)
    train_rmse = mean_squared_error(y_train_reg, y_train_by_average, squared=False)
    test_r2 = r2_score(y_test_reg, y_pred)
    test_mae = mean_absolute_error(y_test_reg, y_pred)
    test_rmse = mean_squared_error(y_test_reg, y_pred, squared=False)
    history = {
        'train_r2': train_r2,
        'train_mae': train_mae,
        'train_rmse': train_rmse,
        'test_r2': test_r2,
        'test_mae': test_mae,
        'test_rmse': test_rmse,
    }

    memory_cleanup()

    combined_history[seed] = history    

# # Display metrics
# display_metrics_with_ci(combined_history)

# save the history as pickle
with open(f'./data/history/baseline_{experiment_name}.pickle', 'wb') as handle:
    pickle.dump(combined_history, handle, protocol=pickle.HIGHEST_PROTOCOL)

# print average results
print('Average results:')
for metric in ['r2', 'mae', 'rmse']:
    train_metric = [combined_history[seed][f'train_{metric}'] for seed in seeds]
    test_metric = [combined_history[seed][f'test_{metric}'] for seed in seeds]
    print(f'{metric.upper()}: mean = {np.mean(test_metric):.4f}, 95% CI = [{np.percentile(test_metric, 2.5):.4f}, {np.percentile(test_metric, 97.5):.4f}]')

end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time:.2f} seconds")

# no attention:
# 1694.4s	178	TEST METRICS FOR THE BEST EPOCH (#10)
# 1694.4s	179	R2: mean = 0.4464, 95% CI = [0.3873, 0.5055]
# 1694.4s	180	MAE: mean = 0.3759, 95% CI = [0.3352, 0.4166]
# 1694.4s	181	RMSE: mean = 0.4848, 95% CI = [0.4521, 0.5175]

AVERAGE


Starting for seed 42...
----------------------------------------------------------------------------------------------------


Starting for seed 78687...
----------------------------------------------------------------------------------------------------


Starting for seed 123123...
----------------------------------------------------------------------------------------------------

Average results:
R2: mean = -0.0001, 95% CI = [-0.0003, -0.0000]
MAE: mean = 0.5130, 95% CI = [0.5122, 0.5143]
RMSE: mean = 0.6286, 95% CI = [0.6280, 0.6290]
Execution time: 1.00 seconds


In [33]:
from scipy.stats import t

def mean_confidence_interval(data, confidence=0.95):
        n = len(data)
        m, se = np.mean(data), np.std(data) / np.sqrt(n)
        h = se * t.ppf((1 + confidence) / 2, n-1)
        return m, m-h, m+h

# get mean and +- 95% CI for the metrics
r2 = [combined_history[seed]['test_r2'] for seed in seeds]
mae = [combined_history[seed]['test_mae'] for seed in seeds]
rmse = [combined_history[seed]['test_rmse'] for seed in seeds]

print(f'R2: mean = {np.mean(r2):.4f}, 95% CI = {mean_confidence_interval(r2)}')
print(f'MAE: mean = {np.mean(mae):.4f}, 95% CI = {mean_confidence_interval(mae)}')
print(f'RMSE: mean = {np.mean(rmse):.4f}, 95% CI = {mean_confidence_interval(rmse)}')


R2: mean = -0.0001, 95% CI = (-0.00014983344452172234, -0.00048374797871964285, 0.00018408108967619814)
MAE: mean = 0.5130, 95% CI = (0.5130321940487806, 0.5106100256899847, 0.5154543624075766)
RMSE: mean = 0.6286, 95% CI = (0.6286108107707062, 0.6274224108817511, 0.6297992106596614)
